# Étape 1 / Exploration de données non structurées

Récolter deux types de données en passant par l’API Binance.  

* Grâce à cette API, on peut aller récupérer des informations sur les cours des différents marchés (BTC-USDT, BTC-ETH, …).
* Le but sera de créer une fonction de récupération de données générique afin de pouvoir avoir les données de n’importe quel marché.

## Explications générales

>**API Terminology**

These terms will be used throughout the documentation, so it is recommended that you read them to enhance your understanding of the API (especially for new users).
- **Base asset** refers to the asset that is the quantity of a symbol; for the symbol BTCUSDT, BTC would be the base asset.
- **Quote asset** refers to the asset that is the price of a symbol; for the symbol BTCUSDT, USDT would be the quote asset.  

>**Symbol Status**
- PRE_TRADING
- TRADING
- POST_TRADING
- END_OF_DAY
- HALT
- AUCTION_MATCH
- BREAK  

>**Status	Description**

`NEW`	The order has been accepted by the engine  
`PARTIALLY_FILLED`	Part of the order has been filled  
`FILLED`	The order has been completed  
`CANCELED`	The order has been canceled by the user  
`PENDING_CANCEL`	This is currently unused  
`REJECTED`	The order was not accepted by the engine and not processed  
`EXPIRED`	The order was canceled according to the order type's rules (e.g., LIMIT FOK orders with no fill, LIMIT IOC, or MARKET orders that partially fill), or by the exchange(e.g., orders canceled during liquidation or orders canceled during maintenance)  
`EXPIRED_IN_MATCH`	The order was canceled by the exchange due to STP. (e.g. an order with EXPIRE_TAKER will match with existing orders on the book with the same account or same tradeGroupId)  

## Données de base des paires

In [30]:
import requests
import pandas as pd

def get_pair_info(pair):
    url = 'https://api.binance.com/api/v3/exchangeInfo'
    params = { 'symbol': pair.upper() }
    data = requests.get(url, params = params).json()

    if not data.get('symbols'):
        return None  # Paire invalide ou absente

    symbol_data = data['symbols'][0]
    filtres = { f['filterType']: f for f in symbol_data['filters'] }

    return {
        'symbol': symbol_data['symbol'],
        'baseAsset': symbol_data['baseAsset'],
        'quoteAsset': symbol_data['quoteAsset'],
        'status': symbol_data['status'],
        'orderTypes': symbol_data['orderTypes'],
        'minPrice': filtres.get('PRICE_FILTER', {}).get('minPrice'),
        'maxPrice': filtres.get('PRICE_FILTER', {}).get('maxPrice'),
        'tickSize': filtres.get('PRICE_FILTER', {}).get('tickSize'),
        'minQty': filtres.get('LOT_SIZE', {}).get('minQty'),
        'maxQty': filtres.get('LOT_SIZE', {}).get('maxQty'),
        'stepSize': filtres.get('LOT_SIZE', {}).get('stepSize'),
        'minNotional': filtres.get('NOTIONAL', {}).get('minNotional'),
        'maxNotional': filtres.get('NOTIONAL', {}).get('maxNotional'),
    }


info = get_pair_info('BTCUSDT')
print(info)


{'symbol': 'BTCUSDT', 'baseAsset': 'BTC', 'quoteAsset': 'USDT', 'status': 'TRADING', 'orderTypes': ['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS', 'STOP_LOSS_LIMIT', 'TAKE_PROFIT', 'TAKE_PROFIT_LIMIT'], 'minPrice': '0.01000000', 'maxPrice': '1000000.00000000', 'tickSize': '0.01000000', 'minQty': '0.00001000', 'maxQty': '9000.00000000', 'stepSize': '0.00001000', 'minNotional': '5.00000000', 'maxNotional': '9000000.00000000'}


###Fonction avec une liste

In [31]:
import requests
import pandas as pd
import json

def get_pair_info(pairs):
    url = 'https://api.binance.com/api/v3/exchangeInfo'
    params = { 'symbols': json.dumps(pairs) } 
    data = requests.get(url, params = params).json()

    if not data.get('symbols'):
        return None  

    pairs_general_info = []

    for symbol_data in data['symbols']:
        filtres = { f['filterType']: f for f in symbol_data['filters'] }

        pairs_general_info.append({
            'symbol': symbol_data['symbol'],
            'baseAsset': symbol_data['baseAsset'],
            'quoteAsset': symbol_data['quoteAsset'],
            'status': symbol_data['status'],
            'orderTypes': symbol_data['orderTypes'],
            'minPrice': filtres.get('PRICE_FILTER', {}).get('minPrice'),
            'maxPrice': filtres.get('PRICE_FILTER', {}).get('maxPrice'),
            'tickSize': filtres.get('PRICE_FILTER', {}).get('tickSize'),
            'minQty': filtres.get('LOT_SIZE', {}).get('minQty'),
            'maxQty': filtres.get('LOT_SIZE', {}).get('maxQty'),
            'stepSize': filtres.get('LOT_SIZE', {}).get('stepSize'),
            'minNotional': filtres.get('NOTIONAL', {}).get('minNotional'),
            'maxNotional': filtres.get('NOTIONAL', {}).get('maxNotional'),
        })

    return pairs_general_info

lst_pairs = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT']
info = get_pair_info('BTCUSDT')

print(info)


None


In [32]:
def display_pair_info(info):
    if info is None:
        print("Paire introuvable.")
        return

    print(f"Pair Informations : {info['symbol']}\n")
    print(f"{'Base asset':15} : {info['baseAsset']}")
    print(f"{'Quote asset':15} : {info['quoteAsset']}")
    print(f"{'Statut':15} : {info['status']}")
    print(f"{'Order types':15} : {', '.join(info['orderTypes'])}")

    print("\nPair Prices Informations :")
    print(f"{'Min price':15} : {info['minPrice']}")
    print(f"{'Max price':15} : {info['maxPrice']}")
    print(f"{'Tick size':15} : {info['tickSize']}")
    print(f"{'Min notional':15} : {info['minNotional']}")
    print(f"{'Max notional':15} : {info['maxNotional']}")

    print("\nPair Quantity Informations :")
    print(f"{'Min qty':15} : {info['minQty']}")
    print(f"{'Max qty':15} : {info['maxQty']}")
    print(f"{'Step size':15} : {info['stepSize']}")

display_pair_info(info)

Paire introuvable.


## Autres données

## Boucle avec toutes les paires pour constuire le dataframe des infos de base

# A garder pour l'étape 3

In [ ]:
# Récupération de toutes les données
url = 'https://api.binance.com/api/v3/exchangeInfo'
data = requests.get(url).json()
pair_list = data['symbols']

# Boucle sur chaque paire avec la fonction
pairs_base_infos = get_pair_info(pair_list)

# Création du DataFrame final
df_pairs_base_infos = pd.DataFrame(pairs_base_infos)

# Affichage
print(df_pairs_base_infos.head())

KeyboardInterrupt: 

In [ ]:
resp = requests.get('https://api.binance.us/api/v3/trades?{BTCUSDT}')

resp.json()

In [ ]:
url = 'https://api.binance.com/api/v3/ticker/24hr'
data = requests.get(url).json()
data

In [ ]:
url = 'https://api.binance.com/api/v3/ticker/price'
response = requests.get(url)
tickers = response.json()

tickers